In [1]:
from keras.applications import VGG16
from keras.datasets import fashion_mnist
from keras.preprocessing.image import img_to_array, array_to_img
from keras import models, layers
from keras import callbacks
import numpy as np
import keras_tuner

In [2]:
#preparing data
num_classes = 10
num_features = 784
(train_data, train_labels), (test_data, test_labels) = fashion_mnist.load_data()
#converting into 3 channels
train_data, test_data = np.stack([train_data] * 3, axis=-1), np.stack([test_data] * 3, axis=-1)
#resizing
train_data = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in train_data])
test_data = np.asarray([img_to_array(array_to_img(im, scale=False).resize((48,48))) for im in test_data])
train_data, test_data = np.array(train_data, np.float32), np.array(test_data, np.float32)
train_data, test_data = train_data / 255., test_data / 255.
print(train_data.shape)

(60000, 48, 48, 3)


In [3]:
#model creator
def create_model():
    vgg = VGG16(
        include_top=False,
        weights="imagenet",
        input_shape=(48, 48, 3))
    vgg.trainable = False
    model = models.Sequential([
        vgg,
        layers.Flatten(),
        layers.Dense(128, activation="relu"),
        layers.Dropout(0.1),
        layers.Dense(10, activation="softmax")
    ])
    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"])
    return model

In [4]:
#building the model
stop_early = callbacks.EarlyStopping(monitor='val_loss', patience=5)  
model = create_model()
model.fit(train_data, train_labels, epochs=20, validation_data=(test_data, test_labels), callbacks=[stop_early])

Epoch 1/20
1875/1875 [==============================] - 2732s 1s/step - loss: 0.5182 - accuracy: 0.8172 - val_loss: 0.4103 - val_accuracy: 0.8532
Epoch 2/20
1875/1875 [==============================] - 2665s 1s/step - loss: 0.3840 - accuracy: 0.8591 - val_loss: 0.3759 - val_accuracy: 0.8667
Epoch 3/20
1875/1875 [==============================] - 2655s 1s/step - loss: 0.3511 - accuracy: 0.8701 - val_loss: 0.3707 - val_accuracy: 0.8647
Epoch 4/20
1875/1875 [==============================] - 2661s 1s/step - loss: 0.3284 - accuracy: 0.8795 - val_loss: 0.3546 - val_accuracy: 0.8737
Epoch 5/20
1875/1875 [==============================] - 2658s 1s/step - loss: 0.3146 - accuracy: 0.8834 - val_loss: 0.3522 - val_accuracy: 0.8727
Epoch 6/20
1875/1875 [==============================] - 2656s 1s/step - loss: 0.3005 - accuracy: 0.8886 - val_loss: 0.3431 - val_accuracy: 0.8759
Epoch 7/20
1875/1875 [==============================] - 2664s 1s/step - loss: 0.2895 - accuracy: 0.8921 - val_loss: 0.3507 -

In [5]:
#getting accuracy
model.evaluate(test_data, test_labels)
#Цікаво що тут точність ще нижче ніж в простій мережі з попереднього завдання.
#Не хочу чекати ще 11 годин поки навчиться модель з іншими параметрами. Вже час здавати це завдання.
#Можливо і можна досягти більшої точності, але я точно не встигну цю модель натренувати до кінця тижня.

313/313 [==============================] - 352s 1s/step - loss: 0.3316 - accuracy: 0.8888


[0.3315577805042267, 0.8888000249862671]